In [1]:
# instsall tar2tf module
! pip3 install ../

Processing /home/tknopik/inf/go/src/github.com/NVIDIA/aistore/python/tar2tf
  Created wheel for tar2tf: filename=tar2tf-0.2.0-py3-none-any.whl size=9563 sha256=3f10678a42bf4dcc8f52fbca01f929209709a226137c4c6438747a97370b3008
  Stored in directory: /tmp/pip-ephem-wheel-cache-u8pqd5y_/wheels/cb/f5/a9/a30b25c3d57be04c840bf44bbf07cf1abf912dbe44934c155e
Successfully built tar2tf


In [3]:
# list available ais buckets
!ais ls ais://

AIS Buckets (1)
  tar-bucket


In [4]:
# list contents of your bucket
!ais ls ais://tar-bucket

NAME		 SIZE		 VERSION	 
train-0.tar	17.48MiB	1		
train-1.tar	17.48MiB	1		
train-10.tar	17.48MiB	1		
train-11.tar	17.48MiB	1		
train-12.tar	17.48MiB	1		
train-13.tar	17.48MiB	1		
train-14.tar	17.48MiB	1		
train-15.tar	17.48MiB	1		
train-16.tar	17.48MiB	1		
train-17.tar	17.48MiB	1		
train-18.tar	17.48MiB	1		
train-19.tar	17.48MiB	1		
train-2.tar	17.48MiB	1		
train-20.tar	17.48MiB	1		
train-3.tar	17.48MiB	1		
train-4.tar	17.48MiB	1		
train-5.tar	17.48MiB	1		
train-6.tar	17.48MiB	1		
train-7.tar	17.48MiB	1		
train-8.tar	17.48MiB	1		
train-9.tar	17.48MiB	1		


In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from tar2tf import AisDataset
from tar2tf.ops import Select, Decode, Convert, Resize

In [7]:
EPOCHS = 5
BATCH_SIZE = 20

# ADJUST AisDataset PARAMETERS BELOW

BUCKET_NAME = "tar-bucket"
PROXY_URL = "http://localhost:8080"

# Create AisDataset.
# Values will be extracted from tar-records according to Resize(Convert(Decode("jpg"), tf.float32), (224, 224)) operation,
# meaning that bytes under "jpg" in tar-record will be decoded as an image, converted to tf.float32 type and then Resized to (224, 224)
# Labels will be extracted from tar-records according to Select("cls") operation, meaning that bytes under "cls" will be treated as label.
conversions = [Decode("jpg"), Convert("jpg", tf.float32), Resize("jpg", (224, 224))]
selections = ["jpg", "cls"]
ais = AisDataset(BUCKET_NAME, PROXY_URL, conversions, selections)

# edit arguments to match your bucket objects
train_dataset = ais.load("train-{0..3}.tar",
                         num_workers=4
).prefetch(BATCH_SIZE * EPOCHS).shuffle(buffer_size=1024).batch(BATCH_SIZE)

test_dataset = ais.load("train-{4..7}.tar"
).prefetch(BATCH_SIZE).batch(BATCH_SIZE)

# TRAINING PART BELOW

REMOTE EXECUTION DISABLED (train-{0..3}.tar)
REMOTE EXECUTION DISABLED (train-{4..7}.tar)


In [8]:
inputs = keras.Input(shape=(224, 224, 3), name="images")
x = layers.Flatten()(inputs)
x = layers.Dense(64, activation="relu", name="dense_1")(x)
x = layers.Dense(64, activation="relu", name="dense_2")(x)
outputs = layers.Dense(10, name="predictions")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

model.compile(optimizer=keras.optimizers.Adam(1e-4), loss=keras.losses.mean_squared_error, metrics=["acc"])
model.summary()

model.fit(train_dataset, epochs=EPOCHS)
result = model.evaluate(test_dataset)
print(dict(zip(model.metrics_names, result)))


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
images (InputLayer)          [(None, 224, 224, 3)]     0         
_________________________________________________________________
flatten (Flatten)            (None, 150528)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                9633856   
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
predictions (Dense)          (None, 10)                650       
Total params: 9,638,666
Trainable params: 9,638,666
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
20/20 [==============================] - 5s 243ms/step - loss: 79920.7842 - acc: 0.9500
Epoch 2/5
20/20 [==

KeyboardInterrupt: 